
$$
\begin{aligned}
\frac{r(s)}{\delta(s)}=G_{\delta}^{r}(0) \frac{1+T_{r} s}{1+\frac{2 \zeta s}{\omega_{n}}+\frac{s^{2}}{\omega_{n}^{2}}}
\end{aligned}
$$

$$
G_{\delta}^{r}(0)=\frac{1}{1+A V^{2}} \frac{V}{l} \\
G(0)=\frac{1}{1+A V^{2}} \frac{V}{Nl}
$$

$$
\begin{aligned}
\zeta=\frac{m\left(l_{f}^{2} K_{f}+l_{r}^{2} K_{r}\right)+I\left(K_{f}+K_{r}\right)}{2 l \sqrt{m I K_{f} K_{r}\left(1+A V^{2}\right)}}
\end{aligned}
$$
$$
\begin{aligned}
T_{r}=\frac{m l_{f} V}{2 l K_{r}}
\end{aligned}
$$
$$
\begin{aligned}
\omega_{n} & =\frac{2 l}{V} \sqrt{\frac{K_{f} K_{r}}{m I}} \sqrt{1+A V^{2}}
\end{aligned}
$$

$$
\begin{gathered}
G_{\delta}^{\beta}(0)=\frac{1-\frac{m}{2 l} \frac{l_{f}}{l_{r} K_{r}} V^{2}}{1+A V^{2}} \frac{l_{r}}{l} 
\end{gathered}
$$


$$
\begin{aligned}
A=-\frac{m}{2 l^{2}} \frac{l_{f} K_{f}-l_{r} K_{r}}{K_{f} K_{r}}
\end{aligned}
$$



In [ ]:

$$
\begin{aligned}
\frac{r(s)}{\delta(s)}=G_{\delta}^{r}(0) \frac{1+T_{r} s}{1+\frac{2 \zeta s}{\omega_{n}}+\frac{s^{2}}{\omega_{n}^{2}}}
\end{aligned}
$$

$$
G_{\delta}^{r}(0)=\frac{1}{1+A V^{2}} \frac{V}{l} \\
G(0)=\frac{1}{1+A V^{2}} \frac{V}{Nl}
$$

$$
\begin{aligned}
\zeta=\frac{m\left(l_{f}^{2} K_{f}+l_{r}^{2} K_{r}\right)+I\left(K_{f}+K_{r}\right)}{2 l \sqrt{m I K_{f} K_{r}\left(1+A V^{2}\right)}}
\end{aligned}
$$
$$
\begin{aligned}
T_{r}=\frac{m l_{f} V}{2 l K_{r}}
\end{aligned}
$$
$$
\begin{aligned}
\omega_{n} & =\frac{2 l}{V} \sqrt{\frac{K_{f} K_{r}}{m I}} \sqrt{1+A V^{2}}
\end{aligned}
$$

$$
\begin{gathered}
G_{\delta}^{\beta}(0)=\frac{1-\frac{m}{2 l} \frac{l_{f}}{l_{r} K_{r}} V^{2}}{1+A V^{2}} \frac{l_{r}}{l} 
\end{gathered}
$$


$$
\begin{aligned}
A=-\frac{m}{2 l^{2}} \frac{l_{f} K_{f}-l_{r} K_{r}}{K_{f} K_{r}}
\end{aligned}
$$



In [3]:
import json
from ipywidgets import interactive, Button, HBox, VBox, IntSlider, FloatSlider, Output
import numpy as np
from matplotlib import ticker
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt


# ---unique--------------------------------
output = Output()

# Function to create an interactive plot
def interactive_plot_func(mf, mr, l, I, N, K_f, K_r, V):
    # Convert units to SI units
    l = l / 1000  # (mm -> m)
    V = V / 3.6  # (kph -> m/s)
    K_f = K_f / np.pi * 180  # (N/deg -> N/rad)
    K_r = K_r / np.pi * 180  # (N/deg -> N/rad)
    m = mf + mr  # Mass
    l_f = l/m*mr  # Front wheel location
    l_r = l/m*mf  # Rear wheel location

    # Calculate other parameters
    A = -(m/(2*l**2)) * (l_f*K_f - l_r*K_r) / (K_f*K_r)
    G_dr_0 = 1/(1+A*V**2) * V/(N*l)
    zeta = (m*(l_f**2*K_f + l_r**2*K_r) + I*(K_f + K_r))/(2*l*np.sqrt(m*I*K_f*K_r*(1+A*V**2)))
    T_r = m*l_f*V/(2*l*K_r)
    omega_n = 2*l/V * np.sqrt(K_f*K_r/(m*I)) * np.sqrt(1+A*V**2)

    # Define transfer function
    num = [G_dr_0*T_r, G_dr_0]
    den = [1/omega_n**2, 2*zeta/omega_n, 1]
    system = signal.TransferFunction(num, den)

    # Frequency range for Bode plot
    frequencies = np.logspace(-2, 2, 500)  # Frequency range: 0.01 to 100



    # Compute Bode plot
    w, mag, phase = signal.bode(system, frequencies)

    # Plot Bode diagram
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))

    ax1.semilogx(w/2/np.pi, mag)  # Convert rad/s to Hz for the frequency axis
    ax1.get_xaxis().set_major_formatter(ticker.ScalarFormatter())  # Change x-axis to scalar notation
    ax1.set_title('Bode plot of the transfer function')
    ax1.set_ylabel('Magnitude [dB]')
    ax1.grid(True)
    ax1.set_xlim([0.1, 10])  # Set x-axis limits for the first plot (Magnitude)
    ax1.set_ylim([-40, 10])

    ax2.semilogx(w/2/np.pi, phase)  # Convert rad/s to Hz for the frequency axis
    ax2.get_xaxis().set_major_formatter(ticker.ScalarFormatter())  # Change x-axis to scalar notation
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Phase [deg]')
    ax2.grid(True)
    ax2.set_xlim([0.1, 10])  # Set x-axis limits for the second plot (Phase)
    ax2.set_ylim([-90, 10])

    plt.tight_layout()
    plt.show()
    
    # Find magnitude at omega_n/(2*np.pi) and 0.2 Hz
    idx_omega_n = np.abs(w/(2*np.pi) - omega_n/(2*np.pi)).argmin()
    idx_0_2Hz = np.abs(w/(2*np.pi) - 0.2).argmin()
    idx_0_5Hz = np.abs(w/(2*np.pi) - 0.5).argmin()
    idx_1Hz = np.abs(w/(2*np.pi) - 1.0).argmin()

    mag_omega_n = mag[idx_omega_n]
    mag_0_2Hz = mag[idx_0_2Hz]
    phase_0_5Hz = phase[idx_0_5Hz]
    phase_1Hz = phase[idx_1Hz]
    ds_ratio = mag_omega_n - mag_0_2Hz
    
    with output:
        output.clear_output()  # Clear the previous value
        print(f"Yaw resonance frequency [Hz]: {omega_n/(2*np.pi):.2f}")  # Print the new value
        print(f"Stability factor [x10^(-3) s^2/m^2]: {A*10**3:.2f}")  # Print the new value
        print(f"Dynamic static ratio [dB]: {ds_ratio:.2f}")
        print(f"Phase at 0.5 [Hz]: {phase_0_5Hz:.2f} deg")
        print(f"Phase at 1.0 [Hz]: {phase_1Hz:.2f} deg")
    display(output)
    
    # Create DataFrame for Bode data
    df_all = pd.DataFrame({'Frequency [Hz]': w/2/np.pi, 'Magnitude [dB]': mag, 'Phase [deg]': phase})
        
    return df_all

def save_data(button):
    df_all = interactive_plot_func(**{child.description.split(' ')[0]: child.value for child in interactive_plot.children[:-1]})
    df_all.to_csv('data.csv', index=False)
    
# ---partially unique--------------------------------

# define a dictionary that links the description to the name and unit
slider_parameters = {
    "mf": {"name": "Front mass", "unit": "kg"},
    "mr": {"name": "Rear mass", "unit": "kg"},
    "l": {"name": "Wheelbase", "unit": "mm"},
    "I": {"name": "Yaw inertia", "unit": "kgm^2"},
    "V": {"name": "Velocity", "unit": "kph"},
    "N": {"name": "Gear ratio", "unit": "-"},
    "K_f": {"name": "Front cornering stiffness", "unit": "N/deg"},
    "K_r": {"name": "Rear cornering stiffness", "unit": "N/deg"},
}

# Creating the sliders
spec_sliders = {
    "mf": IntSlider(min=500, max=2000, step=10, value=1000, description=f'mf [{slider_parameters["mf"]["unit"]}]', continuous_update=False),
    "mr": IntSlider(min=500, max=2000, step=10, value=700, description=f'mr [{slider_parameters["mr"]["unit"]}]', continuous_update=False),
    "l": IntSlider(min=2000, max=4000, step=10, value=2700, description=f'l [{slider_parameters["l"]["unit"]}]', continuous_update=False),
    "I": IntSlider(min=2000, max=4000, step=10, value=2650, description=f'I [{slider_parameters["I"]["unit"]}]', continuous_update=False),
    "N": FloatSlider(min=10, max=20, step=0.5, value=15, description=f'N [{slider_parameters["N"]["unit"]}]', continuous_update=False),
    "K_f": IntSlider(min=500, max=2000, step=100, value=1000, description=f'K_f [{slider_parameters["K_f"]["unit"]}]', continuous_update=False),
    "K_r": IntSlider(min=500, max=2000, step=100, value=1300, description=f'K_r [{slider_parameters["K_r"]["unit"]}]', continuous_update=False),
}
condition_sliders = {
    "V": IntSlider(min=30, max=120, step=30, value=120, description=f'V [{slider_parameters["V"]["unit"]}]', continuous_update=False)
}

# ---common--------------------------------

interactive_plot = interactive(interactive_plot_func,
                               **spec_sliders, 
                               **condition_sliders)

# Function to define the operation of the Save button
def save_values(b):
    spec_slider_values = {
        key: {
            'name': slider_parameters[key]['name'],
            'value': slider.value,
            'unit': slider_parameters[key]['unit']
        }
        for key, slider in spec_sliders.items()
    }
    try:
        with open('spec_defaults.json', 'r+') as f:
            data = json.load(f)
            data.update(spec_slider_values)
            f.seek(0)
            f.write(json.dumps(data))
            f.truncate()
    except FileNotFoundError:
        with open('spec_defaults.json', 'w') as f:
            json.dump(spec_slider_values, f)
            
# Function to define the operation of the Load button
def load_values(b):
    try:
        with open('spec_defaults.json', 'r') as f:
            defaults = json.load(f)
            for key, slider in spec_sliders.items():
                if key in defaults:
                    slider.value = defaults[key]["value"]
    except FileNotFoundError:
        print("No default values found. Please save first.")

# create buttons
save_param_button = Button(description="Save parameters")
save_param_button.on_click(save_values)

load_param_button = Button(description="Load parameters")
load_param_button.on_click(load_values)

save_data_button = Button(description="Save data")
save_data_button.on_click(save_data)

# Displaying the UI elements
display(HBox([interactive_plot, VBox([save_param_button, load_param_button, save_data_button])]))